## 2024년도 도전학기 '빅데이터와 인공지능을 활용한 시스템 강건설계' AI 챌린지

#### - AI 챌린지 목표: 실습내용 기반 데이터 가공 능력, 인공지능 모델 학습 능력 평가
> #### <font color=red>FFT 데이터 기반 1DCNN 고장진단 만들기</font>
#### - 제공 데이터 구성 : 데이터 챌린지에서 사용된 정상 및 고장 용접 데이터
#### - 데이터 저장 시 index = None, header = None 필수!
#### - 총 3단계의 AI 챌린지를 수행하며, 단계별 결과가 저장된 폴더(Result) 내의 결과물을 1개 압축파일(.zip)로 제출
#### - <font color=blue>AI 챌린지에 대한 질의는 "김규민", "이예림", "이현석" 조교에게 아이캠퍼스 메세지를 통해 진행</font>

.

.

.



## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf  
from tensorflow import keras

# [1단계] 데이터 불러오기 & FFT 실행하기(15점)
> #### Data 폴더 내부 정상/고장 각각 150개씩
> #### 데이터는 가속도/전압/전류 3개 센서, 샘플링 주파수 12800Hz, 약 0.2167초간의 데이터
> #### 모든 데이터를 불러와 TotalData(300, 2774, 3)로 만든 후 모든 센서에 대해 FFT 적용
> #### FFT 적용된 데이터 이름은 TotalData_FFT(300, 1387, 3)
> #### <font color=red>★ TotalData, TotalData_FFT 순서: Normal_1, Normal_2, ..., Normal_150, Abnormal_1, Abnormal_2, ..., Abnormal_150</font>
> #### <font color=red>★ TotalData, TotalData_FFT 이름 수정 X

In [2]:
NoOfData   = 150                   # 데이터 개수 (정상/고장 각각)
DataLength = 2774                   # 데이터 길이
NoOfSensor = 3                   # 센서 개수

NoOfData, DataLength, NoOfSensor

(150, 2774, 3)

#### 데이터 불러오기 (TotalData 만들기)

In [3]:
# 전체 데이터 만들기

TotalData = np.zeros([ NoOfData*2, DataLength, NoOfSensor]) # 데이터 빈집

for i in range(NoOfData):
    tempNormalPath   = './Data/Normal_%d.csv'%(i+1)
    tempAbnormalPath = './Data/Abnormal_%d.csv'%(i+1)
    tempNormal   = pd.read_csv(tempNormalPath,header=None).iloc[:,:]
    tempAbnormal = pd.read_csv(tempAbnormalPath,header=None).iloc[:,:]
    TotalData[i, :, :]            = tempNormal
    TotalData[i + NoOfData, :, :] = tempAbnormal

TotalData.shape

(300, 2774, 3)

#### FFT 적용하기 (TotalData_FFT 만들기)
- FFT 결과 중 Amplitude 값 저장하기

In [4]:
Fs         = 12800
FFT_length = 1387

In [5]:
# Compute FFT
TotalData_FFT = np.zeros([NoOfData*2, FFT_length, NoOfSensor])

for i in range(NoOfData*2):
    for j in range(NoOfSensor):
        x = TotalData[i, : , j]
        # FFT 적용 코드 
        NFFT = len(x)
        
        Y = np.fft.fft(x)/NFFT
        Y = Y[range(math.trunc(NFFT/2))]
        amp_Hz = 2*abs(Y)
     
        TotalData_FFT[i, : , j] = amp_Hz
        
TotalData_FFT.shape

(300, 1387, 3)

### 1단계 결과물을 제출용 파일로 저장(수강생 번호 외 코드 수정 X)

In [6]:
StudentNo = 199   # 수강생 번호 입력

# 아래는 수정 금지
Path1 = './Result/ST%d_AC1_1.csv'%StudentNo
Path2 = './Result/ST%d_AC1_2.csv'%StudentNo
Path3 = './Result/ST%d_AC1_3.csv'%StudentNo

pd.DataFrame(TotalData    [102]).to_csv(Path1, header=None, index=None)
pd.DataFrame(TotalData_FFT[118]).to_csv(Path2, header=None, index=None)
pd.DataFrame(TotalData_FFT[213]).to_csv(Path3, header=None, index=None)

# [2단계] 데이터 분할 및 레이블 만들기(10점)
> #### TrainData : ValidData = 4 : 1 (필수: 실습과 동일한 방식으로 분할)
> #### 정상 레이블: [1, 0], 고장 레이블: [0, 1]
> #### <font color=red>★ TrainData, ValidData 이름 수정 X

#### 데이터 분할

In [7]:
NoOfValid  = int(NoOfData/5)    # 정상/고장 별 검증 데이터 개수
NoOftrain  = NoOfData - NoOfValid    # 정상/고장 별 학습 데이터 개수

NoOfData, NoOfValid, NoOftrain

(150, 30, 120)

In [8]:
# 정상 및 고장 별 학습/검증 데이터 분할
Normal   = TotalData_FFT[:NoOfData, :, :]  
Abnormal = TotalData_FFT[NoOfData:, :, :]

Normal_train   = Normal[  :NoOftrain, :, :] 
Abnormal_train = Abnormal[  :NoOftrain, :, :]

Normal_valid   = Normal[  NoOftrain:, :, :]
Abnormal_valid = Abnormal[NoOftrain:, :, :]

TrainData = np.concatenate([Normal_train, Abnormal_train], axis = 0)
ValidData = np.concatenate([Normal_valid, Abnormal_valid], axis = 0)

TrainData.shape, ValidData.shape

((240, 1387, 3), (60, 1387, 3))

#### 레이블 만들기

In [9]:
TrainLabel = np.zeros((NoOftrain*2,2) , dtype=int)
ValidLabel = np.zeros((NoOfValid*2,2) , dtype=int)

TrainLabel[:NoOftrain , 0] = 1    # 정상(학습)데이터 레이블 : [1,0]
TrainLabel[NoOftrain: , 1] = 1    # 고장(학습)데이터 레이블 : [0,1]

ValidLabel[:NoOfValid , 0] = 1    # 정상(검증)데이터 레이블 : [1,0]
ValidLabel[NoOfValid: , 1] = 1    # 고장(검증)데이터 레이블 : [0,1]


TrainLabel = pd.DataFrame(TrainLabel) # 수정 금지
ValidLabel = pd.DataFrame(ValidLabel) # 수정 금지

TrainLabel.shape, ValidLabel.shape

((240, 2), (60, 2))

### 2단계 결과물을 제출용 파일로 저장(수강생 번호 외 코드 수정 X)

In [10]:
# StudentNo = 199  # 수강생 번호 입력

# 아래는 수정 금지
Path1 = './Result/ST%d_AC2_1.csv'%StudentNo
Path2 = './Result/ST%d_AC2_2.csv'%StudentNo
Path3 = './Result/ST%d_AC2_3.csv'%StudentNo
Path4 = './Result/ST%d_AC2_4.csv'%StudentNo

pd.DataFrame(TrainData[191]).to_csv(Path1, header=None, index=None)
pd.DataFrame(ValidData[37 ]).to_csv(Path2, header=None, index=None)
TrainLabel                  .to_csv(Path3, header=None, index=None)
ValidLabel                  .to_csv(Path4, header=None, index=None)

# [3단계] CNN 모델 만들기 (15점)
> ### 모델 구조
> Input layer > 1D Conv. layer 1 > 1D Conv. layer 2 > Max pooling layer > 1D Conv. layer 3 > 1D Conv. layer 4 > Global average pooling layer > Hidden layer 1 > Hidden layer 2 > Output layer
> ### Layer 별 파라미터
> Conv. layer: filter 수 4개, kernel 크기 4, stride 2, same padding, activation func. relu
> 
> Max pooling layer: pooling 크기 3, stride 3
> 
> Hidden layer: 뉴런 수 32, activation func. relu

#### 모델 구조 설계

In [11]:
learningRate  = 0.001 #수정 금지
Epoch         = 100   #수정 금지

In [12]:
def CNN(input_data):
    keras.backend.clear_session()        # 수정 금지
    tf.keras.utils.set_random_seed(777)  # 수정 금지

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(shape=(input_data.shape[1], input_data.shape[2])))     # Input layer
    model.add(keras.layers.Conv1D(filters = 4, kernel_size=4, strides=2, padding='same', activation=keras.activations.relu))   # Convolution layer 1
    model.add(keras.layers.Conv1D(filters = 4, kernel_size=4, strides=2, padding='same', activation=keras.activations.relu))   # Convolution layer 2
    model.add(keras.layers.MaxPooling1D(pool_size = 3, strides=3))      # Max pooling layer
    model.add(keras.layers.Conv1D(filters = 4, kernel_size=4, strides=2, padding='same', activation=keras.activations.relu))   # Convolution layer 3
    model.add(keras.layers.Conv1D(filters = 4, kernel_size=4, strides=2, padding='same', activation=keras.activations.relu))   # Convolution layer 4
    
    model.add(keras.layers.GlobalAveragePooling1D())                                                           # Global Average Pooling
    model.add(keras.layers.Dense(units = 32, activation = keras.activations.relu,    name = 'Hidden1'))        # Hidden Layer 1
    model.add(keras.layers.Dense(units = 32, activation = keras.activations.relu,    name = 'Hidden2'))        # Hidden Layer 2
    model.add(keras.layers.Dense(units = 2, activation=keras.activations.softmax))                             # Output Layer
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate), # 수정 금지
                  loss=keras.losses.CategoricalCrossentropy(),                    # 수정 금지
                  metrics=['accuracy'])                                           # 수정 금지
    return model

#### 모델 학습 (학습과정 "hist"에 저장 필수!)

In [13]:
# 모델 만들기
CNN_model = CNN(TrainData)

CNN_model.summary() # 모델 구조, 파라미터 개수 확인

# 모델 학습
hist = CNN_model.fit(TrainData, TrainLabel, epochs=Epoch, verbose = 1)

Loss, Accuracy = CNN_model.evaluate(TrainData,  TrainLabel, verbose=0)
print('Final Loss and Accuracy: {:.4f}, {:.2f}%'.format(Loss, Accuracy*100))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 694, 4)              │              52 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 347, 4)              │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 115, 4)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 58, 4)               │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 29, 4)               │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 4)                   │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Hidden1 (Dense)                      │ (None, 32)                  │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Hidden2 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,538 (6.01 KB)

 Trainable params: 1,538 (6.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.4341 - loss: 0.6935
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4341 - loss: 0.6933
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4238 - loss: 0.6932
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5271 - loss: 0.6932
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5271 - loss: 0.6931
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5271 - loss: 0.6930
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5271 - loss: 0.6930
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5271 - loss: 0.6930
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5271 - loss: 0.6929
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5271 - loss: 0.6929
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5271 - loss: 0.6929
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5271 - los

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.1367
Epoch 71/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.1178
Epoch 72/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0994
Epoch 73/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0830 
Epoch 74/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0719
Epoch 75/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0620
Epoch 76/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0540
Epoch 77/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0474
Epoch 78/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0418
Epoch 79/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0373
Epoch 80/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0332
Epoch 81/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0

### 3단계 결과물을 제출용 파일로 저장(수강생 번호 외 코드 수정 X)

In [14]:
# StudentNo = 199   # 수강생 번호 입력

# 아래는 수정 금지
Path1 = './Result/ST%d_AC3_1.keras'%StudentNo
Path2 = './Result/ST%d_AC3_2.csv'%StudentNo
Path3 = './Result/ST%d_AC3_3.csv'%StudentNo


CNN_model.save(Path1)                                                                                           
pd.DataFrame(hist.history                                         ).to_csv(Path2, header=None, index=None)
pd.DataFrame(CNN_model.evaluate(ValidData,  ValidLabel, verbose=0)).to_csv(Path3, header=None, index=None)

.

.

.

# [최종] 해당 코드 파일을 최종 실행한 후 Result 폴더에 넣기

### 1. 해당 코드 파일명의 마지막에 본인 수강생 번호로 변경(ex: 수강생 번호 13번일 경우, AI_Challenge_ST-13)
### 2. 모든 코드를 작성한 후 "Kernel" > "Restart Kernel and Run All Cells..."를 수행한 후 Result 폴더에 넣기

# ● 결과가 저장된 폴더(Result) 내의 모든 파일을 하나의 zip파일로 제출
> #### 압축파일 이름 ST(수강생번호)_AC.zip (예시: 한 자리 수 'ST0_AC', 두 자리 수 'ST00_AC', 세 자리 수 'ST000_AC',)
> #### Data 폴더는 압축 필요 없음!
> #### 영상의 가이드와 다른 방식으로 압축할 경우 감점 (필히 AI 챌린지 가이드 영상 확인 !)